# Screening Task for QOSF Mentorship Program

*Dmytro Fedoriaka, October 2025.*

This is my solution to a screening task of cohort 11 of the [Quantum Open Source Foundation](https://qosf.org/) Mentoship Program.

I chose Task 1, "Gate Tomography".

## Problem statement

Given the circuit
 
<img src="circuit1.png" width=600  />

and the definition of $U3$ gate

$$
U3(\theta, \phi, \lambda) = 
\begin{pmatrix}
\cos(\frac{\theta}{2}) & -e^{i \lambda} \sin(\frac{\theta}{2})  \\
e^{i \phi} \sin(\frac{\theta}{2}) & e^{i (\phi + \lambda)} \cos(\frac{\theta}{2})  \\
\end{pmatrix},
$$

find the parameters $\theta_1, \phi_1, \lambda_1, \theta_2, \phi_2, \lambda_2$ such that when in the circuits two "?" gates are replaced with $U3(\theta_1, \phi_1, \lambda_1)$ and $U3(\theta_2, \phi_2, \lambda_2)$, the circuit is equivalent to a Toffoli gate.

## Solution method

Let's insert gates $U3(\theta_1, \phi_1, \lambda_1)$ and $U3(\theta_2, \phi_2, \lambda_2)$ and compute unitary matrix of that circuit, denote it $U_c$. It will be a function of 6 parameters $\mathbf{w}=(\theta_1, \phi_1, \lambda_1, \theta_2, \phi_2, \lambda_2)$. Now, we need to find such a value of parameter vector $\mathbf{w}$ so that $U_c(\mathbf{w})=U_{CCNOT}$, where $U_{CCNOT}$ is the matrix of the Toffoli gate.

We can write this as optimization problem:

$$ \| U_c(\mathbf{w}) - U_{CCNOT} \| \to \underset{\mathbf{w} \in [0, 2\pi]^6}{\min}$$

We chose optimization domain $[0, 2\pi]^6$, because $U3$ is periodic in each parameter with period $2 \pi$. For norm $\|.\|$ let's use L2-norm, i.e. sum of squares of absolute values of elements.

Let's solve this optimization problem using the L-BFGS algorithm from SciPy. If we find argument $\mathbf{w}$ which makes objective function equal to 0, that means we found the solution to the problem.

We'll use Cirq to build circuit and find its unitary matrix.

## Solution

In [1]:
import cirq
import numpy as np
from scipy.optimize import minimize

# Definition of U3 gate.
def U3(theta, phi, lmbda):
    return cirq.MatrixGate(np.array(
        [[np.cos(theta/2), -np.exp(1j*lmbda)*np.sin(theta/2)], 
        [np.exp(1j*phi)*np.sin(theta/2), np.exp(1j*(phi+lmbda))*np.cos(theta/2)]]
    ))

# Definition of circuit.
def circuit(params):
    ct = cirq.Circuit()
    q = cirq.LineQubit.range(3) 
    ct.append(cirq.T.on(q[0]))
    ct.append(U3(params[0], params[1], params[2]).on(q[2]))
    ct.append(cirq.CNOT.on(q[0], q[1]))
    ct.append((cirq.T**-1).on(q[1]))
    ct.append(cirq.CNOT.on(q[0], q[1]))
    ct.append(cirq.T.on(q[1]))
    ct.append(cirq.CNOT.on(q[1], q[2]))
    ct.append((cirq.T**-1).on(q[2]))
    ct.append(cirq.CNOT.on(q[0], q[2]))
    ct.append(cirq.T.on(q[2]))
    ct.append(cirq.CNOT.on(q[1], q[2]))
    ct.append(U3(params[3], params[4], params[5]).on(q[2]))
    ct.append(cirq.CNOT.on(q[0], q[2]))
    ct.append(cirq.T.on(q[2]))
    ct.append(cirq.H.on(q[2]))
    return ct

# Objective function to minimize.
def f(params):
    diff = cirq.unitary(circuit(params)) - cirq.unitary(cirq.CCNOT)
    return np.sum(np.abs(diff)**2)


bounds = [(0, 2*np.pi)] * 6
np.random.seed(0)
x0 = np.random.uniform(0, 2*np.pi, size=6)
result = minimize(f, x0, bounds=bounds, method='L-BFGS-B')
print("Objective:", result.fun)
print("Converged:", result.success)
print("Optimal parameters:", result.x)
print("Optimal parameters divided by pi:", result.x/np.pi)

Objective: 4.550317086325e-12
Converged: True
Optimal parameters: [1.57079707 6.28318531 3.14159347 0.         2.1374513  3.36033484]
Optimal parameters divided by pi: [0.50000024 2.         1.00000026 0.         0.68037188 1.0696278 ]


Solution is found!

Let's look at the results and try writing them down as simply as possible, keeping in mind that these angles are periodic with period $2 \pi$. 

We can see that $\theta_1 = \pi/2, \phi_1 = 0, \lambda_1 = \pi, \theta_2 = 0$.

It seems like $\phi_2$ and $\lambda_2$ are not "round" numbers. However, when $\theta_2=0$, $\sin(\frac{\theta_2}{2})=0$, and $U3(\theta_2, \phi_2, \lambda_2)$ doesn't depend on $\phi_2$ and $\lambda_2$ independently, but depends only on their sum. We see that $\phi_2 + \lambda_2 = 1.75 \pi$, so we can take $\phi_2 = - \pi/4$ and $\lambda_2=0$.

## Verification

In [2]:
theta1, phi1, lambda1 =  np.pi/2, 0.0, np.pi
theta2, phi2, lambda2 =  0.0, -np.pi/4, 0.0
ct = circuit([theta1, phi1, lambda1, theta2, phi2, lambda2])
print(ct)
assert np.allclose(cirq.unitary(ct), cirq.unitary(cirq.CCNOT))
print("OK")

0: ───T─────────────────────────@──────────@──────────────────@─────────────────────────────────────────@───────────
                                │          │                  │                                         │
1: ─────────────────────────────X───T^-1───X───T───@──────────┼───────@─────────────────────────────────┼───────────
                                                   │          │       │                                 │
      ┌                     ┐                      │          │       │   ┌                         ┐   │
2: ───│ 0.707+0.j  0.707-0.j│──────────────────────X───T^-1───X───T───X───│1.   +0.j    0.   -0.j   │───X───T───H───
      │ 0.707+0.j -0.707+0.j│                                             │0.   +0.j    0.707-0.707j│
      └                     ┘                                             └                         ┘
OK


## Answer

The first gate is $U3(\pi/2, 0, \pi)$, the second gate is $U3(0, -\pi/4, 0)$